<a href="https://colab.research.google.com/github/KateMals/AB_testing/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.stats import ttest_1samp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
from google.colab import files
uploaded = files.upload()

Saving результаты А_B.xlsx to результаты А_B.xlsx


In [7]:
data = pd.read_excel('результаты А_B.xlsx')

data.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [9]:
data.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

In [10]:
45489-44700

789

In [11]:
data.userid.nunique()

90189

Количество уникальных userid показывает, что сплитование по версиям произведено корректно и каждый userid принадлежит только одной версии (либо gate_40, либо gate_30) Разница кол-ва в каждой выборке 45489 - 44700 = 789, это не более 5%, поэтому проблем с эти не будет

In [24]:
group_30 = data[data['version'] == 'gate_30']
group_40 = data[data['version'] == 'gate_40']

In [27]:
group_30.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [28]:
group_40.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


Если рассмотреть показатели двух групп, то среднее значение retention_1 в группе gate_30 (44,82%), и retention_7 также в группе gate_30 (19%).

In [22]:
from statsmodels.stats import proportion
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import kstest

In [16]:
n1 = group_30.retention_7.value_counts().sum()
n2 = group_40.retention_7.value_counts().sum()
k1 = group_30.retention_7.value_counts()[1]
k2 = group_40.retention_7.value_counts()[1]

n1,n2,k1,k2

(44700, 45489, 8502, 8279)

In [17]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]),
np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =10.013, pvalue = 0.002


In [18]:
stats, pvalue = proportion.proportions_ztest(np.array([k1, k2]),
np.array([n1, n2]))

print('Results are ','z =%.3f, pvalue = %.3f'%(stats, pvalue))

Results are  z =3.164, pvalue = 0.002


Результаты Хи-теста и z-теста показывают pvalue = 0.002, что означает, что есть статистически значимые различия между группами, и можно считать, что метрика retention_7 лучше себя ведет в группе gate_40.

In [19]:
m1 = group_30.retention_1.value_counts().sum()
m2 = group_40.retention_1.value_counts().sum()
l1 = group_30.retention_1.value_counts()[1]
l2 = group_40.retention_1.value_counts()[1]

m1,m2,l1,l2

(44700, 45489, 20034, 20119)

In [20]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([l1, l2]),
np.array([m1, m2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =3.183, pvalue = 0.074


In [21]:
stats, pvalue = proportion.proportions_ztest(np.array([l1, l2]),
np.array([m1, m2]))

print('Results are ','z =%.3f, pvalue = %.3f'%(stats, pvalue))

Results are  z =1.784, pvalue = 0.074


Результаты Хи-теста и z-теста показывают pvalue = 0.074, что означает, что статистически значимые различия между группами отсутствуют, и можно считать, что метрика retention_1 не показывает себя лучше ни в какой группе и эту метрику для выделения какой-то из групп в лучшую мы не будем брать.

 Так как retention 7 day позволяет оценить, насколько успешно игра удерживает пользователей на протяжении длительного периода времени, и это позволяет оценить эффективность стратегий удержания игроков на протяжении недели.ТО делаем вывод, что нужно выбрать в качестве стратегии вариант с 40- дневным установленным уровнем в игре.